In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["train/NORMAL", "train/PNEUMONIA"]  # Uso de barras normales
imgs = []  # Lista para almacenar las imágenes
labels = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpeg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs = np.array(imgs)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (115, 150, 150)
Num. of labels: 115


In [3]:
# Crear un modelo secuencial correctamente
model = keras.Sequential([
    keras.layers.Input((150, 150, 1)), # Los dos primeros don las dimensiones, el último es la escala de color
    keras.layers.Conv2D(32,5, padding="same", strides=1, activation="relu"), # Empiricamente el profe dice que esta mejor comenzar con 32
    # El padding es para igualar el tamaño de la imágen de salida al de la imágen de entrada
    # El strinde de 1 es recomendable para clasificación
    keras.layers.MaxPooling2D(), # El valor del producto de cada matriz por el filtro que salga tomamos el mas grande
    keras.layers.Conv2D(16,2, padding="same", strides=2),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(16,5, padding="same", strides=1, activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(16,2, padding="same", strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(100),
    keras.layers.Dense(100),
    keras.layers.Dense(2, activation="softmax")
])

# Imprimir el resumen del modelo
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 38, 38, 16)     │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 19, 19, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 19, 19, 16)     │         6,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 9, 9, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 5, 5, 16)       │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        40,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,754 (237.32 KB)

 Trainable params: 60,754 (237.32 KB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
X_train = np.expand_dims(imgs, axis=-1)
print(X_train.shape)

Y_train = np.expand_dims(labels, axis=-1)
print(Y_train.shape)

(115, 150, 150, 1)
(115, 1)


In [5]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=20, batch_size=32)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.6824 - loss: 0.6749
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7050 - loss: 0.6270
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6873 - loss: 0.6311
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6168 - loss: 0.6012
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6449 - loss: 0.5236
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9142 - loss: 0.4018
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9135 - loss: 0.2961
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8815 - loss: 0.2343
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8631 - loss: 0.3069
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9354 - loss: 0.1462
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9649 - loss: 0.1286
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9639 - loss: 0.1306
E

In [6]:
paths = ["test/NORMAL", "test/PNEUMONIA"]  # Uso de barras normales
imgs = []  # Lista para almacenar las imágenes
labels = []  # Lista para almacenar las etiquetas

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpeg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy después del bucle
imgs = np.array(imgs)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (35, 150, 150)
Num. of labels: 35


In [7]:
X_test = np.expand_dims(imgs, axis=-1)
print(X_train.shape)

Y_test = np.expand_dims(labels, axis=-1)
print(Y_train.shape)

(115, 150, 150, 1)
(115, 1)


In [9]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis = 1)
print(((predictions == Y_test).sum()/len(Y_test))*100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1800.0
